In [4]:
import similaritymeasures
import numpy as np
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import pairwise_distances
import time
from clustering.distances.discrete import FastDiscreteFrechetMatrix, FastDiscreteFrechetSparse, \
    earth_haversine, euclidean

Compute distance matrix with efficient Frechet-Distance

In [67]:
def compute_distance_matrix(trajectories, method="Frechet"):
    """
    :param method: "Frechet" or "Area"
    """
    n = len(trajectories)
    dist_m = np.zeros((n, n))
    distance = euclidean
    fdfdm = FastDiscreteFrechetMatrix(distance)
    for i in range(n - 1):
        print(i)
        p = trajectories[i]
        for j in range(i + 1, n):
            q = trajectories[j]
            if method == "Frechet":
                # dist_m[i, j] = similaritymeasures.frechet_dist(p, q)
                dist_m[i, j] = fdfdm.distance(p, q)
            else:
                start = time.time()
                dist_m[i, j] = similaritymeasures.area_between_two_curves(p, q)
                end = time.time()
                print('Operation Time Area: {}'.format(end-start))
            dist_m[j, i] = dist_m[i, j]
    return dist_m

In [68]:
trajs = np.random.randn(300, 120, 3)
start = time.time()
dist_matrix = compute_distance_matrix(trajs)
end = time.time()
print("Total Time : {}".format(end - start))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

Test the efficiency of different Frechet Distances

In [2]:
distance = euclidean
fdfdm = FastDiscreteFrechetMatrix(distance)
sdsd = FastDiscreteFrechetSparse(distance)
t0 = np.random.randn(500, 3)
t1 = np.random.randn(500, 3)
t0 = t0.astype(np.float32)
t1 = t1.astype(np.float32)
print(t0.dtype)
start = time.time()
print(fdfdm.distance(t0, t1))
end = time.time()
print("Total Time : {}".format(end - start))
start = time.time()
print(sdsd.distance(t0, t1))
end = time.time()
print("Total Time : {}".format(end - start))
start = time.time()
print(similaritymeasures.frechet_dist(t0, t1))
end = time.time()
print("Total Time : {}".format(end - start))

float32
3.0097339153289795
Total Time : 0.9428806304931641
3.0097339153289795
Total Time : 1.1464378833770752
3.00973382431566
Total Time : 2.7458174228668213


In [ ]:
clusterer = KMedoids(20, metric='precomputed')
clusterer.fit(dist_matrix)

Pairwise Distances with Custom Distance

In [16]:
distance = euclidean
fdfdm = FastDiscreteFrechetSparse(distance)
trajs = np.random.randn(20, 500, 3)

#all_distances = pairwise_distances([trajs[0]], trajs[1:])

all_distances = [[]]
for tr in trajs[1:]:
    all_distances[0].append(fdfdm.distance(trajs[0], tr))
print(np.shape(all_distances))


(1, 19)
